# Set-up

In [ ]:
# libraries
import re
import numpy as np
import pandas as pd
from pymongo import MongoClient

In [ ]:
# let's connect to the localhost
client = MongoClient()

# let's create a database 
db = client.moma

# collection
artworks = db.artworks

# print connection
print("""
Database
==========
{}

Collection
==========
{}
""".format(db, artworks), flush=True
)

## Data

![MoMa](https://images.musement.com/cover/0001/31/moma-museum-of-modern-art-tickets-tours-jpg_header-30520.jpeg?&q=60&fit=crop)

In [ ]:
df = pd.read_csv('https://media.githubusercontent.com/media/MuseumofModernArt/collection/master/Artworks.csv')

df.info()

# Loading

In [ ]:
%%time 
# slow loading of data
d = {}

for i in df.index:
    d = {
        "_id": str(df.loc[i, "Cataloged"]) + str(df.loc[i, "ObjectID"]),
        "Title": df.loc[i, "Title"],
        "Date": df.loc[i, "Date"],
        "Artist": {
            "Name": df.loc[i, "Artist"],
            "Bio": df.loc[0, "ArtistBio"],
            "Nationality": df.loc[0, "Nationality"],
            "Birth": df.loc[0, "BeginDate"],
            "Death": df.loc[0, "EndDate"],
            "Gender": df.loc[0, "Gender"]
        },
        "Characteristics":{
            "Medium": df.loc[i,'Medium'], 
            "Dimensions": df.loc[i,'Dimensions'],
            "Circumference": df.loc[i,'Circumference (cm)'], 
            "Depth": df.loc[i,'Depth (cm)'], 
            "Diameter": df.loc[i,'Diameter (cm)'], 
            "Height": df.loc[i,'Height (cm)'],
            "Length": df.loc[i,'Length (cm)'], 
            "Weight": df.loc[i,'Weight (kg)'], 
            "Width": df.loc[i,'Width (cm)'], 
            "Seat Height": df.loc[i,'Seat Height (cm)'],
            "Duration": df.loc[i,'Duration (sec.)']
        },
        "Acquisition": {
            "Date": df.loc[i, "DateAcquired"],
            "CreditLine": df.loc[i, "CreditLine"],
            "Number": df.loc[i, "AccessionNumber"]
        },
        "Classification": df.loc[i, "Classification"],
        "Department": df.loc[i, "Department"],
        "URL": df.loc[i, "URL"], 
        "ThumbnailURL": df.loc[i, "ThumbnailURL"]
    }
    artworks.insert_one(d)

In [ ]:
# for further reference https://docs.mongodb.com/manual/reference/command/collStats/
stats = db.command("collstats", "artworks")
s0 = stats.get('size')/10**6

print("""
Namespace: {}

Document Count: {}

Size: {}

""".format(stats.get('ns'), stats.get('count'), s0), flush=True)

## Cleaning

In [ ]:
# get key names
l = []
for i in d.keys():
    try:
        for b in d.get(str(i)).keys():
            l.append(str(i) + '.' + str(b))
    except:
        l.append(i)

In [ ]:
# unset NaN fields
for i in l:
    update = artworks.update_many({str(i):np.nan},{"$unset": {str(i):""}})
    print("""
    Key: {}
    Matched: {}
    Modified: {}
    ------------
    """.format(i, update.matched_count, update.modified_count), flush=True)

In [ ]:
# for further reference https://docs.mongodb.com/manual/reference/command/collStats/
stats = db.command("collstats", "artworks")
s1 = stats.get('size')/10**6

print("""
Namespace: {}

Document Count: {}

Size: {}

Var. Size: {}

""".format(stats.get('ns'), stats.get('count'), s1, round(s0-s1, 2)), flush=True)

## Further Cleaning

In [ ]:
# change data type
update = artworks.update_many({"Date":{"$regex": '^[0-9]*$'}}, [{ "$set": { "Date": { "$toInt": "$Date" } } }])

print("""
    Key: {}
    Matched: {}
    Modified: {}
    ------------
    """.format("Date", update.matched_count, update.modified_count), flush=True)

In [ ]:
# create an array field to store ranges
for i in artworks.find({"Date":{"$regex": '^[0-9]{4}-[0-9]{4}$'}}):
    date = i.get('Date').split('-')
    a = int(date[0])
    b = int(date[1])
    id = i.get('_id')
    update = artworks.update_one({"_id": str(id)},{"$set": {"Date": [a, b]}})
    print(update.matched_count, update.modified_count)
    
for i in artworks.find({"Date":{"$regex": '^[0-9]{4}–[0-9]{4}$'}}):
    date = i.get('Date').split('–')
    a = int(date[0])
    b = int(date[1])
    id = i.get('_id')
    update = artworks.update_one({"_id": str(id)},{"$set": {"Date": [a, b]}})
    print(update.matched_count, update.modified_count)

for i in artworks.find({"Date": {"$regex": '^[0-9]{4}-[0-9]{2}$'}}, {"Date": 1}):
    date = i.get('Date').split('-')
    a = int(date[0])
    b = int(date[0][0] + date[0][1] + date[1])
    id = i.get('_id')
    update = artworks.update_one({"_id": str(id)},{"$set": {"Date": [a, b]}})
    print(update.matched_count, update.modified_count)

for i in artworks.find({"Date": {"$regex": '^[0-9]{4}–[0-9]{2}$'}}, {"Date": 1}):
    date = i.get('Date').split('–')
    a = int(date[0])
    b = int(date[0][0] + date[0][1]+ date[1])
    id = i.get('_id')
    update = artworks.update_one({"_id": str(id)},{"$set": {"Date": [a, b]}})
    print(update.matched_count, update.modified_count)

# perform some further cleaning
for i in artworks.find({"Date":{"$regex": '^c. [0-9]{4}$'}}):
    date = i.get('Date').split(' ')
    b = int(date[1])
    id = i.get('_id')
    update = artworks.update_one({"_id": str(id)},{"$set": {"Date": b}})
    print(update.matched_count, update.modified_count)

In [ ]:
# remove Unknown or n.d.
update = artworks.update_many({"Date": {"$in": ["n.d.", "Unknown", "unknown"]}}, {"$unset": {"Date": ""}})
print("""
    Matched: {}
    Modified: {}
    """.format(update.matched_count, update.modified_count), flush=True)

In [ ]:
for i in artworks.find({"Date": {"$type": "string"}}, {"Date":1}):
    print(i)

# Aggregation and loading

In [ ]:
# collection
artw = db.artw

# print connection
print("""
Database
==========
{}

Collection
==========
{}
""".format(db, artw), flush=True
)

In [ ]:
# df to dict
df.rename(columns={'Duration (sec.)': 'Duration (sec)'}, inplace=True)
dd = df.to_dict('records')

dd[0]

In [ ]:
%%time
# insert array
insert = artw.insert_many(dd)

# define the pipeline
pipeline = [
    {"$project": 
     {
         "_id": {"$concat": ["$Cataloged", {"$toString": "$ObjectID"}]},
         "Title": "$Title",
         "Date": "$Date",
         "Artist": {
             "Name": "$Artist", 
             'Bio': "$ArtistBio",
             'Nationality': "$Nationality",
             "Birth": "$BeginDate",
             "Death": "$EndDate", 
             "Gender": "$Gender",
         },
        "Characteristics":{
            "Medium": '$Medium', 
            "Dimensions": '$Dimensions',
            "Circumference": '$Circumference (cm)', 
            "Depth": '$Depth (cm)', 
            "Diameter": '$Diameter (cm)', 
            "Height": '$Height (cm)',
            "Length": '$Length (cm)', 
            "Weight": '$Weight (kg)', 
            "Width": '$Width (cm)', 
            "Seat Height": '$Seat Height (cm)',
            "Duration": '$Duration (sec)'
        },
        "Acquisition": {
            "Date": "$DateAcquired",
            "CreditLine": "$CreditLine",
            "Number": "$AccessionNumber"
        },
        "Classification": "$Classification",
        "Department": "$Department",
        "URL": "$URL", 
        "ThumbnailURL": "$ThumbnailURL"
     }
    },
    { "$out" : "artw" }
]

# perform the aggregation
agr = artw.aggregate(pipeline)

In [ ]:
# unset field with null values
[artw.update_many({str(i):np.nan},{"$unset": {str(i):""}}) for i in l]